# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [11]:
import pandas as pd



In [12]:
data = pd.read_csv('/datasets/data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [13]:
display(data.groupby('children')['children'].count())                                      # кол-во клиентов по кол-ву детей)
display(data.groupby('dob_years')['dob_years'].count())                                    # кол-во клиентов оп возрастам)
display(data.groupby(['education_id', 'education'])['education_id'].count())               # образование
display(data.groupby('gender')['gender'].count())                                          # пол
display(data.groupby('purpose')['purpose'].count())                                        # цели кредитов
display(data.groupby(['family_status',  'family_status_id'])['family_status_id'].count())  # семейное положение / кол-во

children
-1        47
 0     14149
 1      4818
 2      2055
 3       330
 4        41
 5         9
 20       76
Name: children, dtype: int64

dob_years
0     101
19     14
20     51
21    111
22    183
23    254
24    264
25    357
26    408
27    493
28    503
29    545
30    540
31    560
32    510
33    581
34    603
35    617
36    555
37    537
38    598
39    573
40    609
41    607
42    597
43    513
44    547
45    497
46    475
47    480
48    538
49    508
50    514
51    448
52    484
53    459
54    479
55    443
56    487
57    460
58    461
59    444
60    377
61    355
62    352
63    269
64    265
65    194
66    183
67    167
68     99
69     85
70     65
71     58
72     33
73      8
74      6
75      1
Name: dob_years, dtype: int64

education_id  education          
0             ВЫСШЕЕ                   274
              Высшее                   268
              высшее                  4718
1             СРЕДНЕЕ                  772
              Среднее                  711
              среднее                13750
2             НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
              Неоконченное высшее       47
              неоконченное высшее      668
3             НАЧАЛЬНОЕ                 17
              Начальное                 15
              начальное                250
4             УЧЕНАЯ СТЕПЕНЬ             1
              Ученая степень             1
              ученая степень             4
Name: education_id, dtype: int64

gender
F      14236
M       7288
XNA        1
Name: gender, dtype: int64

purpose
автомобили                                478
автомобиль                                495
высшее образование                        453
дополнительное образование                462
жилье                                     647
заняться высшим образованием              496
заняться образованием                     412
на покупку автомобиля                     472
на покупку подержанного автомобиля        479
на покупку своего автомобиля              505
на проведение свадьбы                     777
недвижимость                              634
образование                               447
операции с жильем                         653
операции с коммерческой недвижимостью     651
операции с недвижимостью                  676
операции со своей недвижимостью           630
покупка жилой недвижимости                607
покупка жилья                             647
покупка жилья для сдачи                   653
покупка жилья для семьи                   641
покупка коммерческой недви

family_status          family_status_id
Не женат / не замужем  4                    2813
в разводе              3                    1195
вдовец / вдова         2                     960
гражданский брак       1                    4177
женат / замужем        0                   12380
Name: family_status_id, dtype: int64

**Вывод**

В стобцах days_employed, total_income есть значения NaN
В days_employed данные представлены в float64
В столбце children есть значения -1 и 20, их сумма составляет меньше 1%, можно убрать
В столбце dob_years есть 0 в 101 строке, аналогично
В education названия уровня образования написан в разных регистрах, нужно привести к одному виду
В gender один клиент не определился с полом, убрать
В purpose много одинаковых целей записаны в разной форме, привести к общему виду(лемматизировать)
В family_status/family_status_id проблем нет


## Шаг 2. Предобработка данных

### Обработка пропусков

In [14]:
def mean_income(df):
    income_mean = df.groupby('income_type')['total_income'].mean()         # средний доход по типу знятости
    unique_total_income = list(set(df['income_type']))                     # список источников дохода
    dict_mean = dict(income_mean)
    for i in unique_total_income:                                          # заполнение пропусков средними значениями
        df.loc[df["income_type"] == i,'total_income'] = df.loc[df["income_type"] == i,'total_income'].fillna(dict_mean[i])
mean_income(data)

data['days_employed'] = data['days_employed'].apply(abs)                   #убираю "-" в стаже

In [15]:
def mean_days_employed(df):
    employed_mean = df.groupby('dob_years')['days_employed'].mean()      # ср. стаж по возрасту
    unique_employed_mean = list(set(df['dob_years']))                    # список возрастов
    for i in unique_employed_mean:                                       # заполнение пропусков средними значениями
        df.loc[df["dob_years"] == i,'days_employed'] = df.loc[df["dob_years"] == i,'days_employed'].fillna(dict(employed_mean)[i])
mean_days_employed(data)

In [16]:
data.loc[data["children"] == 20,'children'] = 2 #сичтаю, что 20 детей опечатка, меняю на 2
data['children'] = data['children'].apply(abs)  #аналогично с -1, убираю знак минус
data = data[data['gender'] != 'XNA']            #убираю клиента без пола
data = data[data['dob_years'] != 0]             #убираю клиентов с возрастом 0

**Вывод**

Пропуски в столбце days_employed заполнены средними значениями стажа для каждого возраста.
В total_income - средними значениями доходов для каждого типа занятости
Пропуски, возможно, появились из-за невнимательности менеждера банка или клиента, при запонении/проверке анкеты

### Замена типа данных

In [17]:
def experience(df):                                        #перевод стажа из дней в года
    if df['income_type'] != 'пенсионер':
        return df['days_employed']  / 365
    else:
        return df['days_employed']  / 24 / 365
data['years_employed'] =data.apply(experience, axis=1)     #применяю ко всей таблице

In [18]:
data['days_employed'] = data['days_employed'].astype(int)  #замена типа данных на целочисленныый
data['years_employed'] = data['years_employed'].astype(int)
data['total_income'] = data['total_income'].astype(int)
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21423 entries, 0 to 21524
Data columns (total 13 columns):
children            21423 non-null int64
days_employed       21423 non-null int64
dob_years           21423 non-null int64
education           21423 non-null object
education_id        21423 non-null int64
family_status       21423 non-null object
family_status_id    21423 non-null int64
gender              21423 non-null object
income_type         21423 non-null object
debt                21423 non-null int64
total_income        21423 non-null int64
purpose             21423 non-null object
years_employed      21423 non-null int64
dtypes: int64(8), object(5)
memory usage: 2.3+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,23
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,11
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,15
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,11
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,38


**Вывод**

Стаж пенсионеров записан неправдоподобно большими числами. При переводе в года получаются сотни лет, если считать, что он записан в днях, как у остальных категорий. Делаю вывод, что стаж записан в часах.
Столбец со стажем и общим доходом перевел в целые числа

### Обработка дубликатов

In [19]:
display(data.groupby(['education_id', 'education'])['education_id'].count())   #образование / кол-во
print()
data['education'] = data['education'].str.lower()                            # образвание - в нижний регистр
display(data.groupby(['education_id', 'education'])['education_id'].count())

education_id  education          
0             ВЫСШЕЕ                   273
              Высшее                   266
              высшее                  4686
1             СРЕДНЕЕ                  770
              Среднее                  708
              среднее                13691
2             НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
              Неоконченное высшее       47
              неоконченное высшее      665
3             НАЧАЛЬНОЕ                 17
              Начальное                 15
              начальное                250
4             УЧЕНАЯ СТЕПЕНЬ             1
              Ученая степень             1
              ученая степень             4
Name: education_id, dtype: int64

education_id  education          
0             высшее                  5225
1             среднее                15169
2             неоконченное высшее      741
3             начальное                282
4             ученая степень             6
Name: education_id, dtype: int64

**Вывод**

Уровень образования клиента бы записан в разных регистрах

### Лемматизация

In [20]:
from pymystem3 import Mystem
from collections import Counter
m = Mystem()
lemmas_out = ''
for purpose in data['purpose'].unique(): # перебор уникальных значений списка целей кредита
    lemmas = m.lemmatize(purpose)        # выделение лемм уникальных значений
    lemmas_out += ''.join(lemmas)
print(lemmas_out)    
print(Counter(lemmas_out.split()))

покупка жилье
приобретение автомобиль
дополнительный образование
сыграть свадьба
операция с жилье
образование
на проведение свадьба
покупка жилье для семья
покупка недвижимость
покупка коммерческий недвижимость
покупка жилой недвижимость
строительство собственный недвижимость
недвижимость
строительство недвижимость
на покупка подержать автомобиль
на покупка свой автомобиль
операция с коммерческий недвижимость
строительство жилой недвижимость
жилье
операция со свой недвижимость
автомобиль
заниматься образование
сделка с подержанный автомобиль
получение образование
автомобиль
свадьба
получение дополнительный образование
покупка свой жилье
операция с недвижимость
получение высокий образование
свой автомобиль
сделка с автомобиль
профильный образование
высокий образование
покупка жилье для сдача
на покупка автомобиль
ремонт жилье
заниматься высокий образование

Counter({'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 

**Вывод**

Выделил леммы, посчитал самые частоповторяющиеся, на их основе выделил основные категории

### Категоризация данных

In [21]:
data['parents'] = 0                                               # столбец статус родителей
data.loc[data['children'] > 0, 'parents'] = 1

In [22]:
from pymystem3 import Mystem                                      # категоризация по цели кредитования
m = Mystem()
def replace(purpose):
    lemmas = m.lemmatize(purpose)
    if 'автомобиль' in lemmas:
        return 'Автомобиль'
    if 'образование' in lemmas:
        return 'Образование'
    if 'недвижимость' in lemmas or 'жилье' in lemmas:
        return 'Недвижимость'
    if 'свадьба' in lemmas:
        return 'Свадьба' 

data['main_purpose']=data['purpose'].apply(replace)              #новый столбец с исправленнымим целями
print(data['main_purpose'].value_counts())

Недвижимость    10792
Автомобиль       4293
Образование      4004
Свадьба          2334
Name: main_purpose, dtype: int64


In [23]:
def child_status(df):
    if df < 1:
        return 'без детей'
    elif df > 2:
        return 'многодетный'
    else:
        return '1-2 ребенка'
     
data['child_status'] = data['children'].apply(child_status)
print(data.groupby(['children', 'child_status'])['child_status'].count())

children  child_status
0         без детей       14079
1         1-2 ребенка      4849
2         1-2 ребенка      2117
3         многодетный       328
4         многодетный        41
5         многодетный         9
Name: child_status, dtype: int64


**Вывод**

Исходя из целей кредита добавил категории каждому клиенту, также - категорию по количеству детей


## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [24]:
debt_child_depend = data.groupby(['parents', 'debt'])['debt'].count() #зависимость дети\просрочка
debt_child_depend_short = data.groupby('parents')['debt'].count()     # кол-во родителей с просрочкой и без
print('Просрочили кредит без детей {:.2%} от общего числа заемщиков без детей или {} человек'.format(debt_child_depend[0][1] / debt_child_depend_short[0], debt_child_depend[0][1]))
print('Просрочили кредит с детьми {:.2%} от общего числа заемщиков с детьми или {} человек'.format(debt_child_depend[1][1] / debt_child_depend_short[1], debt_child_depend[1][1]))

Просрочили кредит без детей 7.51% от общего числа заемщиков без детей или 1058 человек
Просрочили кредит с детьми 9.19% от общего числа заемщиков с детьми или 675 человек


**Вывод**

Есть небольшая разница между количеством клиентов просрочивших кредит среди бездетных и клиентов с детьми, но она незначительная

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [25]:
family_status_debt = data.groupby(['family_status_id',  'debt'])['debt'].count()     #зависимость семейное положение\просрочка
family_status_debt_short = data.groupby(['family_status_id'])['debt'].count()

print('Просрочили кредит не женат / не замужем {: >13.2%}'.format(family_status_debt[4][1] / family_status_debt_short[4]))
print('Просрочили кредит в разводе {: >25.2%}'.format(family_status_debt[3][1] / family_status_debt_short[3]))
print('Просрочили кредит вдовец / вдова {: >20.2%}'.format(family_status_debt[2][1] / family_status_debt_short[2]))
print('Просрочили кредит гражданский брак {: >18.2%}'.format(family_status_debt[1][1] / family_status_debt_short[1]))
print('Просрочили кредит женат / замужем {: >19.2%}'.format(family_status_debt[0][1] / family_status_debt_short[0]))

Просрочили кредит не женат / не замужем         9.76%
Просрочили кредит в разводе                     7.17%
Просрочили кредит вдовец / вдова                6.49%
Просрочили кредит гражданский брак              9.29%
Просрочили кредит женат / замужем               7.52%


**Вывод**

Чаще всего, если клиент не женат / не замужем, то он допускает просточку по платежу.
Вдовец / вдова реже всех допускают такое

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [26]:
low_income = data['total_income'].quantile(.33)            #уровень дохода между средним и низким среди клиентов
high_income = data['total_income'].quantile(.66)           #уровень дохода между средним и высоким среди клиентов

data['income_level'] = 'Средний'
data.loc[data['total_income'] > high_income, 'income_level'] = 'Выше среднего'
data.loc[data['total_income'] < low_income, 'income_level'] = 'Ниже среднего'

income_level_debt = data.groupby(['income_level',  'debt'])['debt'].count()
income_level_debt_short = data.groupby('income_level')['debt'].count()
print('Просрочили кредит с доходом выше среднего {: >14.2%}'.format(income_level_debt['Выше среднего'][1] / income_level_debt_short['Выше среднего']))
print('Просрочили кредит с доходом ниже среднего {: >14.2%}'.format(income_level_debt['Ниже среднего'][1] / income_level_debt_short['Ниже среднего']))
print('Просрочили кредит со средним доходом {: >19.2%}'.format(income_level_debt['Средний'][1] / income_level_debt_short['Средний']))

Просрочили кредит с доходом выше среднего          7.39%
Просрочили кредит с доходом ниже среднего          8.16%
Просрочили кредит со средним доходом               8.74%


**Вывод**

Хуже всего дела с возвратом кредита всрок обстоят у клиентов со средним доходом, у клиентов с доходом выше среднего - наоборот

- Как разные цели кредита влияют на его возврат в срок?

In [27]:
main_purpose_debt = data.groupby(['main_purpose',  'debt'])['debt'].count()     #зависимость семейное цуль кредита\просрочка
main_purpose_debt_short = data.groupby('main_purpose')['debt'].count()

print('Просрочили кредит на автомобиль {: >22.2%}'.format(main_purpose_debt['Автомобиль'][1] / main_purpose_debt_short['Автомобиль']))
print('Просрочили кредит на недвижимость {: >20.2%}'.format(main_purpose_debt['Недвижимость'][1] / main_purpose_debt_short['Недвижимость']))
print('Просрочили кредит на оборазование {: >20.2%}'.format(main_purpose_debt['Образование'][1] / main_purpose_debt_short['Образование']))
print('Просрочили кредит на свадьбу {: >25.2%}'.format(main_purpose_debt['Свадьба'][1] / main_purpose_debt_short['Свадьба']))

Просрочили кредит на автомобиль                  9.32%
Просрочили кредит на недвижимость                7.22%
Просрочили кредит на оборазование                9.24%
Просрочили кредит на свадьбу                     7.88%


**Вывод**

Кредит на автомобиль возвращеют хуже всего, почти как на образование. Клиенты с кредииом на недвидимость самые надёжные

## Шаг 4. Общий вывод

Самый ответственный клиент - это бездетный вдовец / вдова с доходом выше среднего с целью кредита - на недвижимость. И , если клиент не женат / не замужем со средним доходом и детьми, вероятность просрочка кредит на автомобиль будет выше всех остальных 